Imports

In [86]:
import pandas as pd
import os
import pygsheets
import pydrive
import json
import csv
from google.oauth2 import service_account
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=631554413201-g551prrnk9qi5ck6iejdpoua0q9k7p6q.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


## Preparation
1. Enable google sheet and drive [api](https://developers.google.com/sheets/api/quickstart/python)
2. Place ```client_secrets.json``` in working directory

Get sheet id

In [87]:
# replace here
sheet_id = '1k_gtDTg27ZmP9plBcq5coMw-67EgkIFMFTRHphU9Owg'

client = pygsheets.authorize(client_secret='client_secrets.json')

sheet_data = client.sheet.get(sheet_id)

sheet = client.open_by_key(sheet_id)

print(sheet)

<Spreadsheet 'Copy of Application form MuST Bursaries 2021 (Responses)' Sheets:1>


Select specific worksheet

In [88]:
wks = sheet.worksheet_by_title('Form responses 1')
print(wks)


<Worksheet 'Form responses 1' index:0>


In [89]:
wks.cols # To view the number of columns
wks.rows # To view the number of rows
print(f"There are {wks.cols} columns in the gsheet!")
print(f"There are {wks.rows} rows in the gsheet!")


There are 20 columns in the gsheet!
There are 211 rows in the gsheet!


sheet as dataframe

In [90]:
df = wks.get_as_df()

df.head()


,Timestamp,Name and Surname:,"UJ notes and university ranking\nhttps://cwur.org/2019-2020.php (NWU 922)\nIf not found there, consult http://www.webometrics.info\n(NWU 1788)",Citizenship:,Email address:,Phone number/s:,"What is your highest completed qualification, and from which institution?",What were your final marks/results for your highest qualification?,"Are you currently busy with a qualification? If so, please specify qualification and institution.",What type of qualification are you interested in pursuing?,"Why are you interested in working with MuST, specifically?",What are your strengths?,"And your weaknesses? (Come on, we all have them.)",Please finalise your application by e-mailing your CV to must.assistant@gmail.com. (A brief CV is fine),Is there anything interesting that we should know about you?,Institution/degree credible?\n1 excellent\n2 workable\n3 don't know\n4 no,Scientific background?\n1 well-aligned\n2 eng/science\n3 not appropriate\n4 Unsure\n5 No,Programming experience?\n1 Yes incl Python\n2 Yes\n3 Unsure\n4 No,1 Yes\n2 Maybe\n3 Unsure\n4 No,Motivation and other notes
0,11/08/2020 16:33:13,Dylan testing,,Nope,Sorry,All the numbers,I have a bachelors from the school of hard knocks,I think this should be in another format....,NOpe,Masters,Because there is a form,I know how to type,I do not know what to type,Confirm: CV sent.,None,None,None,None,None,None
1,12/08/2020 11:01:26,Damian Tamine,,RSA,damiantamine@gmail.com,711349659,Matric - St. Benedict's College,78.4 %,B.Eng Mechanical - NWU,Masters,MuST is developing research in field's that ap...,Team-work; problem solving; people skills; per...,Coding (not a mechanical engineer's favorite),Confirm: CV sent.,I participated in the 2018 Varsity Cup Hockey ...,2 NWU,2 - Mechanical,3,"2 1st yr module Programming for Engineers, wit...","Mechanical engineering, programming unsure, go..."
2,12/08/2020 11:22:40,Jacques Ehlers,,South African,jacquesehlers15@gmail.com,826627782,National Senior Certificate,83,"Industrial Engineering, North-West University",Masters,My final-year project is application developme...,"Programming, mathematics and application devel...",Underachiever (I don't take something on unles...,Confirm: CV sent.,I am an adventurous person. I have a small unr...,2 NWU,2 - industrial,1,2,"Industrial engineering, good marks, enjoys app..."
3,12/08/2020 11:39:41,OBAKENG DITHATE,,SOUTH AFRICAN,evans.obakeng@gmail.com,763138495,Matric. Zeerust Hoerskool,70%,yes. BEng Mechanical Engineering. NWU potchefs...,Masters,I have always wanted to learn more about netwo...,I am focused and dedicated. I always make sure...,Personal issues can sometimes distract me from...,Confirm: CV sent.,My final year I don't have any funding for my ...,2 NWU,2 - Mechanical,3,2,"Mechanical engineering, programming unsure, go..."
4,12/08/2020 12:43:20,JP van Deventer,,South-African,jpvd7991@gmail.com,842311071,3rd Year Computer and Electronic Engineering (...,63.5% Average Mark,B.Eng Computer and Electronic Engineering 4th ...,Masters,A common problem with most multi-cultural work...,"Analytical Thinking, Good Communication skills...",Impatience when projects pass the set deadline...,Confirm: CV sent.,I am a very well disciplined individual that e...,2 NWU,1,2,2,"OK marks, not selling himself well in CV."


Get all values google docs folders, and links to each.

In [95]:
name_list = []
id_list = []
link_list = []
for file_list in drive.ListFile({'q': "'1rQjBcmqhDkngnbmn12JNQTPFx6lB3OLF' in parents and trashed=false", 'maxResults': 10}):
  print('Received %s files from Files.list()' % len(file_list)) # <= 10
  for file1 in file_list:
      name_list.append(file1['title'])
      id_list.append(file1['id'])
      link_list.append(f"https://drive.google.com/drive/folders/{file1['id']}?usp=sharing")
      print('title: %s, id: %s' % (file1['title'], file1['id']))

Received 10 files from Files.list()
title: Copy of Application form MuST Bursaries 2021 (Responses), id: 1k_gtDTg27ZmP9plBcq5coMw-67EgkIFMFTRHphU9Owg
title: Application form MuST Bursaries 2021 (Responses), id: 11Eptg94-AU_jLnGVmQLSeMM6ws2JDDPNnyEYrEFYtSo
title: Tawele O, id: 10N4HXLkR_zzYgLpn5xBTOCEb5i7L54nF
title: Thamoethata TK, id: 1JUeAsyDYUD0bRV2JFbYJXmTQI_IvbrgC
title: Wamba ET, id: 103-6wt76eEVuvSjbS_jAt77ondxq91fD
title: Copy of Application form MuST Bursaries 2021, id: 1oEk-bRVInlJZZxC8uAzoN8bFmbgIJ6klsUbKX4q80w4
title: Toukam S, id: 1PUL2Kp9iNMLrBZ8GwnD9LRUzpNxGY3R_
title: Phorah K, id: 1S8K2acdupPIuGOowzlbTUeE_oH_I4-Ya
title: Tondji ICT, id: 1HUtHkkz-00KnHpzv5RrXbkn3IryDj7ll
title: Mashile K, id: 1m5tNC7LwkrPTXBUDiTPQCmLvCOt1GF5Q
Received 10 files from Files.list()
title: Kibari K, id: 1wsyVvSjl_01O70s5PEZj7p7OtN50ppw8
title: Kotanmi GB, id: 1Ve15bEDV6tKtX41zSVPuK2wUGPs7Y_2E
title: Application form MuST Bursaries 2021, id: 1Wui8IONxzB8YDpxPFgM7ENjrx8DDBKZW9QEN_nLmygc
title:

Find and replace name with hyperlinks to gdrive
Replace specific values in gsheets

In [100]:
for name,link in zip(name_list,link_list):
    # print(name, link)

    first = name.split()[0]
    cells = wks.find(first, searchByRegex=False, matchCase=False,
         matchEntireCell=False, includeFormulas=True,
        cols=(1,3), rows=None, forceFetch=True)

    if len(cells) > 1:
        print(f"[WARNING] Duplicate found, Do manually, {name}")
        continue
    elif len(cells) == 0:
        print(f'[WARNING] entry not found, {name}')
        continue

    original_name = cells[0].value
    wks.update_value(cells[0].label, f'=HYPERLINK("{link}", "{original_name}")')


[WARNING] entry not found, Copy of Application form MuST Bursaries 2021 (Responses)
[WARNING] entry not found, Application form MuST Bursaries 2021 (Responses)
[WARNING] entry not found, Copy of Application form MuST Bursaries 2021
[WARNING] entry not found, Application form MuST Bursaries 2021
[WARNING] Duplicate found, Do manually, Olaniyan J
[WARNING] Duplicate found, Do manually, Ilunga G
[WARNING] Duplicate found, Do manually, Ojo OE
[WARNING] Duplicate found, Do manually, Anike MAS
[WARNING] Duplicate found, Do manually, Daniel L
[WARNING] Duplicate found, Do manually, Oosthuizen MC
[WARNING] entry not found, Brochure MuST support 2021
[WARNING] Duplicate found, Do manually, Oosthuizen A
[WARNING] Duplicate found, Do manually, Ali JT
[WARNING] entry not found, Ndlovu L
[WARNING] entry not found, Abolaji BH
[WARNING] Duplicate found, Do manually, Saadio CL
[WARNING] entry not found, Ntumba E
[WARNING] entry not found, brochure template br_dot_03422
[WARNING] entry not found, MuST 